In [ ]:
!pip install ultralytics
from ultralytics import YOLO
import cv2
import numpy as np
import json
import os
from collections import defaultdict

try:
    from databricks import dbutils
except ImportError:
    print("dbutils not available - assuming this is not a Databricks environment")

class ArmMovementTracker:
    def __init__(self, movement_threshold=13, frame_memory=15):
        self.trackers = {}
        self.movement_threshold = movement_threshold
        self.frame_memory = frame_memory

    def get_tracker(self, person_id):
        if person_id not in self.trackers:
            self.trackers[person_id] = {
                'prev_keypoints': {'left': {'wrist': None, 'elbow': None}, 'right': {'wrist': None, 'elbow': None}},
                'movement_counter': {'left': 0, 'right': 0},
                'active_action': None,
                'keypoint_history': {'left': [], 'right': []},
                'action_active': {'left': False, 'right': False}
            }
        return self.trackers[person_id]

    def calculate_keypoint_movement(self, person_id, side, wrist_pos, elbow_pos, frame_count):
        tracker = self.get_tracker(person_id)
        prev_keypoints = tracker['prev_keypoints']
        history = tracker['keypoint_history'][side]
        
        if frame_count % 3 == 0:
            current_keypoints = {'wrist': wrist_pos, 'elbow': elbow_pos, 'frame': frame_count}
            history.append(current_keypoints)
            if len(history) > self.frame_memory:
                history.pop(0)
            prev_keypoints[side]['wrist'] = wrist_pos
            prev_keypoints[side]['elbow'] = elbow_pos
        return history

    def check_significant_movement(self, history):
        if len(history) < 2:
            return False
        total_movement = 0
        for i in range(len(history) - 1):
            curr = history[i]
            next_frame = history[i + 1]
            wrist_movement = np.sqrt((curr['wrist'][0] - next_frame['wrist'][0])**2 + (curr['wrist'][1] - next_frame['wrist'][1])**2)
            elbow_movement = np.sqrt((curr['elbow'][0] - next_frame['elbow'][0])**2 + (curr['elbow'][1] - next_frame['elbow'][1])**2)
            total_movement += (wrist_movement + elbow_movement) / 2
        avg_movement = total_movement / (len(history) - 1)
        return avg_movement > self.movement_threshold

    def update_and_check_movement(self, person_id, side, wrist_pos, elbow_pos, frame_count):
        tracker = self.get_tracker(person_id)
        history = self.calculate_keypoint_movement(person_id, side, wrist_pos, elbow_pos, frame_count)
        if frame_count % 3 == 0 and len(history) >= 2:
            if self.check_significant_movement(history):
                tracker['movement_counter'][side] += 1
                if tracker['movement_counter'][side] >= self.frame_memory:
                    tracker['action_active'][side] = True
            else:
                tracker['movement_counter'][side] = 0
                tracker['action_active'][side] = False
        return tracker['action_active'][side]

def calculate_iou_rotated(points1, points2):
    pts1 = np.array(points1, dtype=np.float32)
    pts2 = np.array(points2, dtype=np.float32)
    x_min = min(np.min(pts1[:, 0]), np.min(pts2[:, 0]))
    y_min = min(np.min(pts1[:, 1]), np.min(pts2[:, 1]))
    x_max = max(np.max(pts1[:, 0]), np.max(pts2[:, 0]))
    y_max = max(np.max(pts1[:, 1]), np.max(pts2[:, 1]))
    w = int((x_max - x_min) * frame_width)
    h = int((y_max - y_min) * frame_height)
    if w <= 0 or h <= 0:
        return 0.0
    mask1 = np.zeros((h, w), dtype=np.uint8)
    mask2 = np.zeros((h, w), dtype=np.uint8)
    pts1_mask = (pts1 - [x_min, y_min]) * [w, h]
    pts2_mask = (pts2 - [x_min, y_min]) * [w, h]
    cv2.fillPoly(mask1, [pts1_mask.astype(np.int32)], 1)
    cv2.fillPoly(mask2, [pts2_mask.astype(np.int32)], 1)
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return intersection / union if union > 0 else 0.0

# Load models with GPU support
pose_model = YOLO("yolo11n-pose.pt")

# Folder containing videos
video_folder = "/Workspace/Users/gautham.prakashan@antstack.io/opart-pose/Videos-Service"
output_folder = "/Workspace/Users/gautham.prakashan@antstack.io/opart-pose/Videos-Final-Service/"  # New output folder
video_extensions = (".mp4", ".avi", ".mov")

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load all video files from the folder
video_paths = [
    os.path.join(video_folder, f)
    for f in os.listdir(video_folder)
    if f.lower().endswith(video_extensions)
]

if not video_paths:
    print(f"No videos found in folder: {video_folder}")
    exit()

print(f"Found {len(video_paths)} videos: {video_paths}")

# Initialize trackers
movement_tracker = ArmMovementTracker(movement_threshold=7, frame_memory=15)

# Load class labels
with open('class.json', 'r') as f:
    class_data = json.load(f)
    categories = {cat['id']: cat['name'] for cat in class_data['categories']}

# Load object coordinates
boxes = []
print("Loading object coordinates")
with open('cord.txt', 'r') as f:
    for line in f:
        values = list(map(float, line.strip().split()))
        class_id = int(values[0])
        points = [(values[i], values[i + 1]) for i in range(1, len(values)-1, 2)]
        xs = [p[0] for p in points]
        ys = [p[1] for p in points]
        x_center = sum(xs) / len(xs)
        y_center = sum(ys) / len(ys)
        width = max(xs) - min(xs)
        height = max(ys) - min(ys)
        boxes.append({
            'class_id': class_id,
            'x_center': x_center,
            'y_center': y_center,
            'width': width,
            'height': height,
            'points': points,
            'label': categories.get(class_id, f"Class_{class_id}")
        })
        print(f"Loaded polygon: class_id={class_id}, points={points}")

# Local temporary CSV file
local_csv_path = "file:/Workspace/Users/gautham.prakashan@antstack.io/opart-pose/tmp/tracked_actions_service_actions_final.csv"
dbfs_csv_path = "dbfs:/tmp/tracked_actions.csv"

# Add CSV header if local file doesn't exist
if not os.path.exists('/Workspace/Users/gautham.prakashan@antstack.io/opart-pose/tmp/tracked_actions_service_actions_final.csv'):
    with open('/Workspace/Users/gautham.prakashan@antstack.io/opart-pose/tmp/tracked_actions_service_actions_final.csv', "w") as f:
        f.write("frame_count,person_id,action,num_people\n")
    print(f"Created {'Workspace/Users/gautham.prakashan@antstack.io/opart-pose/tmp/tracked_actions_service_actions_final.csv'} with header")

# Open CSV file once and keep it open for appending
csv_file = open('/Workspace/Users/gautham.prakashan@antstack.io/opart-pose/tmp/tracked_actions_service_actions_final.csv', "a")

# Process videos in a loop
for video_path in video_paths:
    print(f"Processing video: {video_path}")
    
    # Video setup
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        continue
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Output video setup
    output_video_path = os.path.join(output_folder, f"output_{os.path.basename(video_path)}")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # Process video
    frame_count = 0
for pose_result in pose_model.track(video_path, stream=True, classes=[0], tracker="bytetrack.yaml"):
    frame_count += 1
    frame = pose_result.orig_img  # Get the original frame
    if frame_width == 0 or frame_height == 0:
        frame_height, frame_width = frame.shape[:2]

    # Tracking and pose: Get bounding boxes, track_ids, and keypoints
    tracked_people = {}
    if pose_result.boxes is not None and pose_result.boxes.id is not None:
        for box, track_id, keypoints in zip(pose_result.boxes.xyxy.cpu().numpy(),
                                          pose_result.boxes.id.cpu().numpy(),
                                          pose_result.keypoints.data.cpu().numpy()):
            x1, y1, x2, y2 = map(int, box)
            center = ((x1 + x2) / 2, (y1 + y2) / 2)
            tracked_people[int(track_id)] = {
                'box': (x1, y1, x2, y2),
                'center': center,
                'keypoints': keypoints
            }
            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        print(f"Frame {frame_count}: Tracked {len(tracked_people)} people")

    # Pose processing: Filter valid keypoints and compute centers
    poses = []
    for track_id, data in tracked_people.items():
        keypoints = data['keypoints']
        valid_points = keypoints[keypoints[:, 2] > 0.5, :2]
        if len(valid_points) > 0:
            kp_center = np.mean(valid_points, axis=0)
            poses.append({
                'track_id': track_id,
                'keypoints': keypoints,
                'center': (kp_center[0], kp_center[1])
            })
    print(f"Frame {frame_count}: Detected {len(poses)} poses")

    # Assign PIDs based on sorted track IDs
    current_pids = {}
    pid = 0
    for pose in sorted(poses, key=lambda x: x['track_id']):
        track_id = pose['track_id']
        current_pids[pid] = {'track_id': track_id, 'pose': pose}
        keypoints = pose['keypoints']

        # Draw keypoints
        for i, (x, y, conf) in enumerate(keypoints):
            if conf > 0.5:
                cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)

        # Action detection
        body_part_boxes = []
        for arm_side, wrist_idx, elbow_idx in [('left', 9, 7), ('right', 10, 8)]:
            if keypoints[wrist_idx][2] > 0.65 and keypoints[elbow_idx][2] > 0.65:
                wrist = keypoints[wrist_idx][:2]
                elbow = keypoints[elbow_idx][:2]
                is_action_active = movement_tracker.update_and_check_movement(pid, arm_side, wrist, elbow, frame_count)
                print(f"Person {pid}, {arm_side} arm active: {is_action_active}")

                min_x = max(0, min(wrist[0], elbow[0]) - 20)
                min_y = max(0, min(wrist[1], elbow[1]) - 20)
                max_x = min(frame_width, max(wrist[0], elbow[0]) + 20)
                max_y = min(frame_height, max(wrist[1], elbow[1]) + 20)
                arm_points = [
                    (min_x / frame_width, min_y / frame_height),
                    (max_x / frame_width, min_y / frame_height),
                    (max_x / frame_width, max_y / frame_height),
                    (min_x / frame_width, max_y / frame_height)
                ]
                body_part_boxes.append({
                    "box": [min_x, min_y, max_x, max_y],
                    "points": arm_points,
                    "name": f"P{pid}_{arm_side}_arm",
                    "hand": arm_side,
                    "person_id": pid
                })
                # Draw arm box
                color = (0, 255, 255) if is_action_active else (255, 0, 0)
                cv2.rectangle(frame, (int(min_x), int(min_y)), (int(max_x), int(max_y)), color, 2)

        # Check interactions with objects
        person_interactions = defaultdict(set)
        for box in boxes:  # Assume 'boxes' is defined elsewhere with object detections
            for body_part in body_part_boxes:
                wrist_idx = 9 if body_part['hand'] == 'left' else 10
                if keypoints[wrist_idx][2] > 0.5:
                    iou = calculate_iou_rotated(body_part["points"], box['points'])
                    print(f"IoU for {body_part['name']} with {box['label']}: {iou}")
                    if iou > 0.05:
                        person_id = body_part['person_id']
                        arm_side = body_part['hand']
                        tracker = movement_tracker.get_tracker(person_id)
                        if tracker['action_active'][arm_side]:
                            person_interactions[person_id].add(box['label'])

        # Draw interactions and log to CSV
        for person_id, interacted_objects in person_interactions.items():
            x1, y1, x2, y2 = tracked_people[current_pids[person_id]['track_id']]['box']
            for obj_label in interacted_objects:
                action = f"P{person_id} working with {obj_label}"
                cv2.putText(frame, action, (x1, y1 - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
                csv_line = f"{frame_count},{person_id},{action},{len(tracked_people)}\n"
                csv_file.write(csv_line)
                csv_file.flush()
                print(f"Logged to CSV: {csv_line.strip()}")

        pid += 1

    # Cleanup for this video
    cap.release()
    out.release()  # Release the VideoWriter
    print(f"Finished processing and saved video: {output_video_path}")

    # Move the local CSV to DBFS after processing each video
    try:
        dbutils.fs.cp(f"{local_csv_path}", dbfs_csv_path)
        print(f"Copied {local_csv_path} to {dbfs_csv_path}")
    except NameError:
        print("Not in Databricks environment - keeping local file at", local_csv_path)

# Close the CSV file and final cleanup
csv_file.close()
cv2.destroyAllWindows()

# Check if the file exists and print its contents
if os.path.exists(local_csv_path.replace("file:", "")):
    print(f"File {local_csv_path} exists!")
    with open('/Workspace/Users/gautham.prakashan@antstack.io/opart-pose/tmp/tracked_actions_service_actions_final.csv', "r") as f:
        print("Contents of CSV:")
        print(f.read())
else:
    print(f"File {local_csv_path} not found!")